In [1]:
from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

# Introduzione all'equazione di Fokker-Planck

## Generalità

L'equazione di Fokker-Planck (**FP**) è un'equazione differenziale alle
derivate parziali che descrive l'evoluzione temporale della funzione di
densità di probabilità (*PDF*) di un'osservabile (solitamente la
posizione) associata ad un sistema dinamico eccitato da un processo
random. Dato un processo di Itô in più dimensioni l'equazione
differenziale stocastica (*SDE*) che descrive tale processo è:
$$d\mathbf{X}_t = \boldsymbol{\mu}(\mathbf{X}_t,t)dt + \boldsymbol{\sigma}(\mathbf{X}_t,t)\,d\mathbf{W}_t$$
dove $\mathbf{X}_t$ è un vettore random $N$-dimensionale,
$\boldsymbol{\mu}(\mathbf{X}_t,t)$ un vettore $N$-dimensionale,
$\boldsymbol{\sigma}(\mathbf{X}_t,t)$ è una matrice $N\times M$ e
$\mathbf{W}_t$ è un processo di Wiener standard $M$-dimensionale.\
La densità di probabilità $u(\mathbf{x},t)$ per $\mathbf{X}_t$ che
soddisfa l'equazione di *FP* associata alla *SDE* sarà:
$$\frac{\partial u(\mathbf{x},t)}{\partial t} = -\sum_{i=1}^N \frac{\partial}{\partial x_i} \left[ \mu_i(\mathbf{x},t) u(\mathbf{x},t) \right] + \sum_{i=1}^{N} \sum_{j=1}^{N} \frac{\partial^2}{\partial x_i \, \partial x_j} \left[ D_{ij}(\mathbf{x},t) u(\mathbf{x},t) \right]$$
con $\boldsymbol{\mu} = (\mu_1,\ldots,\mu_N)$ vettore di drift e **D**
tensore di diffusione con espressione:
$$\mathbf{D}=\frac{1}{2}\boldsymbol{\sigma\sigma}^\mathsf{T} = D_{ij}(\mathbf{x},t) = \frac{1}{2}\sum_{k=1}^M \sigma_{ik}(\mathbf{x},t) \sigma_{jk}(\mathbf{x},t).$$
La condizione di normalizzazione per la PDF con condizioni iniziali
$u_0=u(\mathbf{x}_0, 0)$ sarà: $$\int u(\textbf{x}) d\textbf{x}=1$$

## Sistema non lineare di Van der Pol

L'oscillatore di Van der Pol è un oscillatore con un dumping non-lineare
che evolve in accordo con la seguente equazione differenziale ordinaria
del secondo ordine: $$\label{ode1}
    \frac{d^2x}{dt^2}-a(1-x^2)\frac{dx}{dt}+x=0$$ dove il parametro $a$
indica l'intensità del dumping e determina la forma del ciclo limite.
L'equazione sopra è riscrivibile come un sistema di *ODE* del primo
ordine: $$\begin{cases}
    \dot x =y\\
    \dot y = a(1-x^2)y-x
\end{cases}$$ La Fokker-Planck di un oscillatore di *VdP* soggetto a un
processo random è la seguente: $$\label{FPVdP}
    \frac{\partial u}{\partial t}=\frac{\partial}{\partial x}(-g(y) u)+\frac{\partial}{\partial y}(f(x, y) u)+D\frac{\partial^2}{\partial y^2}u$$
L'equazione [\[FPVdP\]](#FPVdP){reference-type="eqref"
reference="FPVdP"} è un'equazione differenziale alle derivate parziali
*parabolica* dove sono presenti due funzioni di drift, $g(y)$, $f(x, y)$
e un termine di diffusione dato dal coefficiente (*diffusivo* $D$) della
derivata seconda. Nel caso in esame le funzioni per i termini di drift e
diffusivi sono: $$\label{drift}
g(y)=y \quad  \quad f(x, y)=(x-a(1-x^2)y) \quad \quad  D=\text{costante} \quad$$
Per il termine di drift $f(x,y)$ il parametro $a$ è stato valutato in
due scenari differenti:

-   $\textt{costante}: \quad a=0.1$;

-   $\text{variabile}: \quad a(t)=a_0\cos(\omega t) \quad \text{con} \quad a_0 = 0.1 \quad \text{e} \quad \omega=0.1$;

Affinché lo schema abbia senso il coefficiente di diffusione deve essere
$D>0$.

# Discretizzazione e metodi di risoluzione

Essendo la **FP** una *PDE* è possibile risolverla analiticamente solo
in alcuni casi particolari. L'obiettivo del seguente lavoro è quello di
studiare i risultati numerici ottenuti applicando alcuni schemi di
discretizzazione ad un'equazione di FP in 2D per un oscillatore di Van
Der Pol. Esistono diversi modi per discretizzare una PDE, qui si è fatto
uso delle differenze finite ed in particolare sono stati utilizzati e/o
valutati i metodi: *esplicito*, *implicito*, *Crank-Nicholson* e *ADI:
Alternating-Direction Implicit*.

## Discretizzazione

Prima di introdurre i metodi utilizzati è necessario definire il campo
su cui verranno valutati i vari operatori discretizzati. Definita la
regione $R$ di interesse come la parte di piano tale per cui vale:
$$R = [-2\pi, 2\pi] \times [-2\pi, 2\pi]$$ è necessario scegliere un
ricoprimento o griglia per poter lavorare su $R$.\
La griglia è definita scegliendo $\Delta x$ e $\Delta y$ (spesso è
conveniente scegliere $\Delta x = \Delta y$) o alternativamente $N_x$ e
$N_y$ (numero di punti per $x$ e $y$). Per identificare i punti si usano
coppie ordinate di indici $(i,j)$ che fanno riferimento al punto
$(x, y)=(i\Delta x, j \Delta y)$ in $R$, con $i = 0, ..., N_x$ e
$j = 0, ..., N_y$. Una funzione $v = v(x,y,t)$ approssimata sul punto
griglia $(i,j)$ e all'$n$-esimo step temporale sarà indicata con
$u_{ij}^n$.

![image](images/Fig.3.PNG){width="9cm"}

## Metodo esplicito

L'uso di un metodo esplicito, come il Forward Time Centered Space
(*FTCS*), consente di risolvere direttamente l'equazione differenziale
alle derivate parziali senza incorrere in costi computazionali elevati.
Il valore $u^{n+1}_{i, j}$ può essere calcolato direttamente
(*esplicitamente*) da $u^n_{i j}$. Chiaramente un'approssimazione così
semplice nasconde problemi di stabilità. Inoltre l'accuratezza di un
*FTCS* è $O(\Delta t)+O(\Delta x^2)$. Di seguito la discretizzazione per
l'equazione [\[FPVdP\]](#FPVdP){reference-type="eqref"
reference="FPVdP"}: $$\begin{split}
    \frac{u^{n+1}_{i,j}-u^{n}_{i,j}}{\Delta t}= u^{n}_{i,j} \left [ -\frac{g(x+dx, y)-g(x-dx, y)}{2\Delta x} + \frac{f(x, y+dy)-f(x, y-dy)}{2\Delta y}\right] \\+ g(x, y) \left[ \frac{u^{n}_{i+1,j}-u^{n}_{i-1,j}}{2\Delta x}\right]+ f(x, y)\left[\frac{u^{n}_{i,j+1}-u^{n}_{i,j-1}}{2\Delta y}\right] + D\left[ \frac{u^n_{i, j+1}-2u^n_{i, j}+u^n_{i, j-1}}{\Delta y^2} \right]
    \end{split}$$ che può essere riscritta come: $$\begin{split}
    u^{n+1}_{i, j}=u^n_{i, j}-\beta_x \left [g^{n}_{i+1, j}u^n_{i+1, j} - g^n_{i-1, j}u^n_{i-1, j}\right] +\beta_y\left[f^n_{i, j+1}u^n_{i, j+1}-f^n_{i, j-1}u^n_{i, j-1}\right] \\ +\alpha_y \left[u^n_{i, j+1}-2u^n_{i, j}+u^n_{i, j-1}\right]
\end{split}$$ Le funzioni $g^{n}_{i, j}$ e $f^{n}_{i, j}$ indicano le
due funzioni di drift allo step temporale $n$ e i coefficienti sono:
$$\begin{aligned}
\alpha_y=\frac{D \Delta t}{\Delta y^2}; \quad \beta_x=\frac{\Delta t}{2\Delta x}; \quad \beta_y=\frac{\Delta t}{2\Delta y};\end{aligned}$$
La condizione iniziale (*CI*) è una distribuzione Gaussiana 2D con
$L=2\pi$ estensione spaziale del reticolo
$$u_0(x, y, 0)=e^{-((x-s\cdot L)^2+(y-s\cdot L)^2)}$$ La costante
$s=0.3$ corrisponde ad uno shift in entrambe le direzioni spaziali,
introdotto nella *CI* per fare in modo che la Gaussiana non sia centrata
in $0$ e mostrare, in questo modo, come venga attratta verso il ciclo
limite dell'oscillatore. Di seguito sono riportati alcuni plot ottenuti
dalla simulazione con metodo FTCS a vari step temporali.

![Slice temporali della FP integrata con il metodo esplicito.\
L'estensione del reticolo è $N=200$.](images/time_exp_0.png){#explicit
width="\\textwidth"}

![Slice temporali della FP integrata con il metodo esplicito.\
L'estensione del reticolo è
$N=200$.](images/time_exp_10000.png){#explicit width="\\textwidth"}

![Slice temporali della FP integrata con il metodo esplicito.\
L'estensione del reticolo è
$N=200$.](images/time_exp_20000.png){#explicit width="\\textwidth"}

![Slice temporali della FP integrata con il metodo esplicito.\
L'estensione del reticolo è
$N=200$.](images/time_exp_45000.png){#explicit width="\\textwidth"}

Fisicamente le scale caratteristiche della diffusione devono essere
inferiori allo step spaziale. Ciò rappresenta un problema se siamo
interessati a scale $\lambda\gg \Delta x$, poiché lo schema *FTCS*
risulta essere inaccurato per lunghezze d'onda corte
($k \Delta x \ll 1$) per via della condizione sulla stabilità dello
schema stesso. Una possibile soluzione è usare un metodo implicito che
smorza le piccole scale (*grandi $k$*) ma che risulta accurato come
$\bigO({\Delta t})+\bigO({\Delta x^2})$ oppure utilizzare uno schema che
preserva le piccole scale ma che integra le grandi scale con
fluttuazioni spurie (*CN*).

## Metodo implicito

Uno schema *fully implicit* è più stabile rispetto ad uno esplicito. Un
esempio è il *BTCS* che nel caso in esame
[\[FPVdP\]](#FPVdP){reference-type="eqref" reference="FPVdP"} prevede la
seguente discretizzazione: $$\begin{split}
  u^{n}_{i, j}=u^{n+1}_{i, j}+\beta_x \left[g^{n+1}_{i+1, j}u^{n+1}_{i+1, j} - g^{n+1}_{i-1, j}u^{n+1}_{i-1, j}\right] \\ -\beta_y\left[f^{n+1}_{i, j+1}u^{n+1}_{i, j+1}-f^{n+1}_{i, j-1}u^{n+1}_{i, j-1}\right] \\ -\alpha_y \left[u^{n+1}_{i, j+1}-2u^{n+1}_{i, j}+u^{n+1}_{i, j-1}\right]
  \end{split}$$ E' chiaro, dunque, che per ottenere $u$ allo step $n$ è
necessario risolvere un sistema di equazioni lineari dove a destra è
presente $u$ allo step temporale $n+1$.\
Riscrivendo la discretizzazione per evidenziare i vari elementi di
matrice si ottiene: $$\begin{split}
    u^n_{i, j}=u^{n+1}_{i, j}(1+2\alpha_y)+u^{n+1}_{i+1, j}\beta_x g^{n+1}_{i+1, j}-u^{n+1}_{i-1, j}\beta_x g^{n+1}_{i-1, j} \\ +u^{n+1}_{i, j+1}(-\beta_y f^{n+1}_{i, j+1}-\alpha_y)+u^{n+1}_{i, j-1}(\beta_y f^{n+1}_{i, j-1}-\alpha_y)
    \end{split}$$ Le matrici associate alle due variabili sono
*tridiagonali*, per cui la loro inversione risulta semplice. Di seguito
si riporta l'espressione matriciale dello schema: $$M_x=
    \begin{bmatrix}
      1&\beta_x g^{n+1}_{i+1, j}&0&...&0\\
      -\beta_x g^{n+1}_{i-1, j}&1&\beta_x g^{n+1}_{i+1, j}&...&0\\
      0&\beta_x g^{n+1}_{i-1, j}&1&...&0\\
      ...&...&...&1&...
    \end{bmatrix}_{N-1\times N-1}$$ $$M_y=
    \begin{bmatrix}
    (1+2\alpha_y) & -\beta_y f^{n+1}_{j+1}-\alpha_y&...&...&0\\
    \beta_y f^{n+1}_{j-1}-\alpha_y &(1+2\alpha_y)& -\beta_y f^{n+1}_{j+1}-\alpha_y&...&0\\
    0 & \beta_y f^{n+1}_{j-1}-\alpha_y & (1+2\alpha_y)&...&0\\
    ...& ... & ... & (1+2\alpha_y)& ...
    \end{bmatrix}_{N-1 \times N-1}$$\
Una volta definiti 2 array bidimensionali, essi vengono aggiornati
iterativamente per le 2 variabili. Il metodo risulta incondizionatamente
stabile. Il principale svantaggio per uno schema puramente implicito,
oltre al fatto di dover risolvere $N-1$ equazioni algebriche, con $N$
numero di step spaziali in entrambe le dimensioni, è l'accuratezza,
$O(\Delta t)+O(\Delta x^2)$.

## Metodo Crank-Nicolson

Lo schema Crank-Nicolson (*CN*) è un metodo alle differenze finite
ottenuto facendo un *averaging* tra uno schema implicito ed uno
esplicito. È uno schema implicito nel tempo con accuratezza pari a
$O(\Delta t^2)+O(\Delta x^2)$, ed è incondizionatamente stabile.
Ciononostante la soluzione approssimata può contenere oscillazioni
spurie (smorzate) se il rapporto $\frac{\Delta t D}{\Delta x^2}$ è molto
grande (maggiore di 1/2 secondo l'analisi di stabilità di Von Neumann).
Per questo motivo, quando sono necessari time steps più grandi o una
risoluzione spaziale migliore, altri metodi sono preferibili anche se
con un'accuratezza minore. Di seguito la discretizzazione di
[\[FPVdP\]](#FPVdP){reference-type="eqref" reference="FPVdP"} attraverso
lo schema $\theta-Method$: $$\begin{split}
\frac{u^{n+1}_{i, j}-u^{n}_{i, j}}{\Delta t} = \theta \biggl[ -\frac{g^{n+1}_{i+1, j} u^{n+1}_{i+1, j}-g^{n+1}_{i-1, j} u^{n+1}_{i-1, j}}{2\Delta x} + \frac{f^{n+1}_{i, j+1} u^{n+1}_{i, j+1}-f^{n+1}_{i, j-1} u^{n+1}_{i, j-1}}{2\Delta y}+ \\
D \frac{u^{n+1}_{i, j+1}-2 u^{n+1}_{i, j}+u^{n+1}_{i, j-1}}{\Delta y^2} \biggr] +(1-\theta) \biggl[-\frac{g^{n}_{i+1, j} u^{n}_{i+1, j}-g^{n}_{i-1, j} u^{n}_{i-1, j}}{2\Delta x} + \\
\frac{f^{n}_{i, j+1} u^{n}_{i, j+1}-f^{n}_{i, j-1} u^{n}_{i, j-1}}{2\Delta y}+D \frac{u^{n}_{i, j+1}-2 u^{n}_{i, j}+u^{n}_{i, j-1}}{\Delta y^2} \biggr]
\end{split}$$ dove lo schema dipende da valore di $\theta$:

-   se $\theta = 0 \to$ *Explicit (FTCS)*;

-   se $\theta = 1/2 \to$ *Crank-Nicolson (CN)*;

-   se $\theta = 1 \to$ *Fully Implicit*;

\
per cui lo schema
$\textt{CN} = \frac{1}{2}(\textit{Fully Implicit})+\frac{1}{2}(\textit{Explicit})$.\
Uno schema di $CN$ in una dimensione restituisce matrici tridiagonali,
la cui inversione risulta numericamente efficiente. In 2 o più
dimensioni lo schema perde di efficienza dato che le matrici sono sparse
e ciò rende laboriosa la risoluzione del sistema di equazioni algebriche
associate. Si può ovviare a questo inconveniente utilizzando tecniche di
*split operator*, come l'*ADI* descritto nel seguito.

## Alternate Direction Implicit Method

L'*Alternate Direction Implicit Method*, o *ADI* è uno schema alle
differenze finite in cui gli step in ogni direzione sono risolti
separatamente, e in ciascuno step si costruisce una struttura implicita
per una direzione spaziale ed esplicita per l'altra. Il maggior
vantaggio di una tecnica di *split operator* come l'*ADI*, è che le
matrici per la parte implicita in ciascuna direzione sono tridiagonali,
per cui l'inversa può essere calcolata facilmente con l'algoritmo di
Thomas, ad esempio. Di seguito si riportano i due steps necessari per
discretizzare la **FP** oggetta di studio
[\[FPVdP\]](#FPVdP){reference-type="eqref" reference="FPVdP"}.\
Come primo step si considera l'implicito su $x$ e l'esplicito su $y$:
$$\label{1ststep}\begin{split}
        \frac{u^{n+1/2}_{i, j}-u^{n}_{i, j}}{\Delta t/2}=-(g^{n+1/2}_{i+1, j}u^{n+1/2}_{i+1, j}-g^{n+1/2}_{i-1, j}u^{n+1/2}_{i-1, j})\frac{1}{2\Delta x}+\\ 
        (f^{n}_{i, j+1}u^{n}_{i, j+1}-f^{n}_{i, j-1}u^{n}_{i, j-1})\frac{1}{2\Delta y}
        +D\frac{u^{n}_{i, j+1}-2u^{n}_{i, j}+u^{n}_{i, j-1}}{\Delta y^2} 
        \end{split}$$ Dopo che l'espressione
[\[1ststep\]](#1ststep){reference-type="eqref" reference="1ststep"} è
stata valutata si procede a far aggiornare l'array delle soluzioni
attraverso lo step successivo, implicito su $y$ ed esplicito su $x$:
$$\label{2ndstep}\begin{split}
    \frac{u^{n+1}_{i, j}-u^{n+1/2}_{i, j}}{\Delta t/2}=-(g^{n+1/2}_{i+1, j}u^{n+1/2}_{i+1, j}-g^{n+1/2}_{i-1, j}u^{n+1/2}_{i-1, j})\frac{1}{2\Delta x}+\\
    (f^{n+1}_{i, j+1}u^{n+1}_{i, j+1}-f^{n}_{i, j-1}u^{n+1}_{i, j-1})\frac{1}{2\Delta y}+D\frac{u^{n+1}_{i, j+1}-2u^{n}_{i, j}+u^{n+1}_{i, j-1}}{\Delta y^2} 
    \end{split}$$ dove le funzioni di drift sono state definite in
[\[drift\]](#drift){reference-type="eqref" reference="drift"}.\
Di seguito si riportano alcuni snapshot dell'evoluzione della PDF a vari
step temporali, integrata con il metodo appena descritto.

![Alcune slice temporali della **FP** integrata con il metodo *ADI*.
L'estensione del reticolo è $N=400$. In questo caso la maggiore
stabilità del metodo ha consentito di costruire una meshgrid più fitta e
ottenere un risultato più preciso. Nell'ultima figura è riportato lo
stesso grafico visto dall'alto per evidenziare il ciclo
limite.](images/time_0.png){#ADI width="\\textwidth"}

![Alcune slice temporali della **FP** integrata con il metodo *ADI*.
L'estensione del reticolo è $N=400$. In questo caso la maggiore
stabilità del metodo ha consentito di costruire una meshgrid più fitta e
ottenere un risultato più preciso. Nell'ultima figura è riportato lo
stesso grafico visto dall'alto per evidenziare il ciclo
limite.](images/time_32500.png){#ADI width="\\textwidth"}

![Alcune slice temporali della **FP** integrata con il metodo *ADI*.
L'estensione del reticolo è $N=400$. In questo caso la maggiore
stabilità del metodo ha consentito di costruire una meshgrid più fitta e
ottenere un risultato più preciso. Nell'ultima figura è riportato lo
stesso grafico visto dall'alto per evidenziare il ciclo
limite.](images/time_45500.png){#ADI width="\\textwidth"}

![Alcune slice temporali della **FP** integrata con il metodo *ADI*.
L'estensione del reticolo è $N=400$. In questo caso la maggiore
stabilità del metodo ha consentito di costruire una meshgrid più fitta e
ottenere un risultato più preciso. Nell'ultima figura è riportato lo
stesso grafico visto dall'alto per evidenziare il ciclo
limite.](images/time_45500_limit_cycle.png){#ADI width="\\textwidth"}

## Caso di drift periodico

Una volta costruito lo schema *ADI* è stato possibile inserire una
funzione periodica nel drift della funzione $f(x,y,t)$, ottenendo come
risultato una *PDF* che si contrae ed espande sul ciclo limite con una
certa frequenza $\omega=0.1$. In particolare il drift periodico è stato
introdotto nel parametro dell'attenuazione non lineare *a*:
$$f(x, y, t)=x-a_0\cos(\omega t)y(1-x^2) \quad \quad \omega=0.1 \quad \quad a_0=0.1$$
Nello schema di integrazione non cambia nulla.

![Alcune slices temporali della **FP** integrata con il metodo *ADI*,
con $N=400$ e drift periodico.](images/time_cos_1800.png){#ADI periodic
width="\\textwidth"}

![Alcune slices temporali della **FP** integrata con il metodo *ADI*,
con $N=400$ e drift periodico.](images/time_cos_34200.png){#ADI periodic
width="\\textwidth"}

![Alcune slices temporali della **FP** integrata con il metodo *ADI*,
con $N=400$ e drift periodico.](images/time_cos_42000.png){#ADI periodic
width="\\textwidth"}

![Alcune slices temporali della **FP** integrata con il metodo *ADI*,
con $N=400$ e drift periodico.](images/time_cos_49800.png){#ADI periodic
width="\\textwidth"}

## Confronto con il ciclo limite

Di seguito si effettua un rapido confronto tra la soluzione dell'*ODE*
in equazione [\[ode1\]](#ode1){reference-type="eqref" reference="ode1"}
e la soluzione della Fokker-Planck associata, con lo stesso parametro
che regola il dumping, $a=2$. L'*ODE* è la seguente: $$\label{ODE}
    \frac{d^2 x}{dt^2}=a(1-x^2)\frac{dx}{dt}-x$$ L'equazione
differenziale è stata risolta con la libreria `solve_ivp` di scipy che
utilizza uno schema `Runge-Kutta al IV ordine`. Di seguito i grafici a
confronto:

![Sinistra: ciclo limite soluzione di
[\[ODE\]](#ODE){reference-type="eqref" reference="ODE"}. Destra:
evoluzione della *PDF* tramite *ADI* dopo 18000 steps
temporali.](images/limit_cycle_ODE.png){#ADI periodic
width="\\textwidth"}

![Sinistra: ciclo limite soluzione di
[\[ODE\]](#ODE){reference-type="eqref" reference="ODE"}. Destra:
evoluzione della *PDF* tramite *ADI* dopo 18000 steps
temporali.](images/limit_cycle_a_2_.png){#ADI periodic
width="\\textwidth"}

## Condizioni iniziali random

Di seguito un esempio di come l'imposizione di una condizione iniziale
in cui tutti gli elementi dell'array bidimensionale $u(x, y, 0)$ sono
presi random da una distribuzione uniforme, faccia evolvere il sistema
sul ciclo limite:

![Esempio di come una CI random venga attratta sul ciclo limite
stabile.](images/random_IC_0.png){#ADI periodic width="\\textwidth"}

![Esempio di come una CI random venga attratta sul ciclo limite
stabile.](images/random_IC_30.png){#ADI periodic width="\\textwidth"}

![Esempio di come una CI random venga attratta sul ciclo limite
stabile.](images/random_IC_90.png){#ADI periodic width="\\textwidth"}

![Esempio di come una CI random venga attratta sul ciclo limite
stabile.](images/random_IC_3000.png){#ADI periodic width="\\textwidth"}

# Convergenza, consistenza e stabilità

Le equazioni differenziali alle derivate parziali sono di rado
risolvibili analiticamente. Una volta approssimata la *PDE* è possibile
conoscere in che modo l'equazione alle differenze finite approssima
l'equazione di partenza, ma questo non fornisce informazioni su quanto
le due soluzioni siano differenti. È chiaro, inoltre, che l'evoluzione
temporale dell'equazione alle differenze finite dal punto di vista
numerico introduca degli errori che possono accumularsi step dopo step.
È possibile definire l'*errore* come la differenza tra la soluzione
esatta del problema originale e la soluzione dell'approsimazione alle
differenze finite ricavata numericamente, cioè se
$\mathbf{U}=[U_1, U_2, ..., U_n]^T$ è la soluzione approssimata generata
dallo schema utilizzato (escludendo errori di *round-off*) e
$\mathbf{v}=[v(x_1), v(x_2), ..., v(x_n)]$ è la soluzione esatta nei
punti $(x_1, x_2, ..., x_n)$, l'errore globale sarà la quantità
$\mathbf{E=U-v}$. Sia $Pv=f$ una *PDE* per l'incognita $v$ definita in
una regione di $\mathbb{R}$, è possibile definire la nozione di
convergenza,consistenza e stabilità per lo schema alle differenze finite
utilizzato:

-   lo schema alle differenze finite $P_i^n u_i^n = F_i^n$ con $u_i^n$
    soluzione approssimata, è definito **convergente** punto a punto
    $\forall (x,t)$ se per $(i\Delta x, (n+1)\Delta t)$ che converge a
    $(x,t)$, $u_i^n$ converge a $v(t)$ con $(\Delta x, \Delta t) \to 0$;

-   lo schema sarà convergente di ordine $(p,q)$ se
    $[(n+1)\Delta t \to t]$ con $(\Delta x, \Delta t) \to 0$ e vale la
    seguente relazione:
    $$\norm{\mathbf{u}^{n+1} - \mathbf{v}^{n+1}} = \bigO(\Delta x^p) + \bigO(\Delta t ^q)$$

-   lo schema alle differenze finite è definito **consistente** se:
    $$\lim_{h\to 0}T(x)=\lim_{h \to 0}(P_h u-Pv)=0$$

-   lo schema alle differenze finite è detto schema **stabile** rispetto
    alla norma $\norm{\cdot}$ se esistono delle costanti positive
    $\Delta x_0$ e $\Delta t_0$ e delle costanti non negative $K$ e
    $\beta$ tali che:
    $$\norm{\mathbf{u}^{n+1}} \leq K e^{\beta t} \norm{\mathbf{u}_0}$$
    per $0 \leq t = (n+1)\Delta t$, $0<\Delta x< \Delta x_0$ e
    $0<\Delta t< \Delta t_0$.\
    \
    Da notare che questa definizione di stabilità non esclude la
    possibilità che la soluzione dell'equazione discretizzata cresca con
    il tempo.

-   **teorema di equivalenza di Lax**: uno schema alle differenze finite
    a 2 livelli consistente per un problema ai valori iniziali ben
    definito è convergente se e solo se è stabile.
    $$consistenza+stabilita \Leftrightarrow convergenza$$

È possibile effettuare un'analisi di stabilità in modo analitico. Uno
schema risulta stabile alla Von Neumann se l'errore non viene
amplificato durante l'evoluzione. Per analizzare l'errore è necessario
decomporre la soluzione discretizzata in serie di Fourier. Per il caso
$2D$ la forma della soluzione sarà la seguente:
$$u_{sp}^{n} = \xi^n e^{[ik (s \Delta x) + il  (p\Delta y)]}$$ con $k,l$
vettori d'onda in ciascuna direzione spaziale e $\xi$ fattore di
amplificazione che impone il vincolo per lo schema sulla
discretizzazione in $\Delta t$ e $\Delta x$. Affinchè lo schema risulti
stabile deve valere $\abs{\xi} \le 1$. La definizione di stabilità
esposta è riferita a schemi con termini omogenei, ma la consistenza e la
stabilità di uno schema omogeneo sono sufficienti per provare la
stabilità dello stesso schema non omogeneo.\
Nel caso preso in esame nessuna delle precedenti definizioni è
direttamente applicabile poiché non si ha a disposizione una soluzione
analitica.

## Errore e stabilità

Nel caso in esame l'errore è stato calcolato numericamente come segue:

-   è stato definito un array i cui elementi sono le spaziature
    temporali $dt$ prese tra $2\cdot10^{-3}$ e $2\cdot 10^{-2}$;

-   variando la spaziatura temporale, occorre modificare anche il
    massimo numero di step temporali per ogni simulazione, in quanto il
    sistema evolve con tempi diversi. Per permettere al sistema di
    evolvere sempre allo stesso modo, nelle simulazioni è stata
    mantenuta costante la quantità $dt\ctod N_t=200$ con $N_t$ numero
    massimo di step temporali; nel caso in cui vengano considerate più
    slices temporali, occorre che durante le varie simulazioni ciascuna
    di esse venga analizzata nello stesso istante (ciò è fondamentale
    poiché il sistema evolve più rapidamente per $dt$ più grandi);

-   per l'errore è stato considerato il massimo della differenza tra il
    valore assunto da $u_{i,j}^n$ nelle varie slice temporali e il
    valore di $u_{i,j}^{n_{dt_0}}$ ad una slice di riferimento fissata
    al valore di $dt$ più piccolo:
    $$\sigma = \max \left |u^n_{i, j}-u^{n_{dt_0}}_{i, j} \right |, \quad u^{n_{dt_0}}_{i,j}\quad t.c. \quad dt_0=2\cdot 10^{-3}, \quad N_t=100000$$

-   l'analisi è stata effettuata in 2 modi differenti:

    -   è stato calcolato l'errore soltanto sull'ultima slice temporale
        dopo aver fatto evolvere il sistema su lunghi tempi;

    -   è stato calcolato l'errore su più istanti temporali per ogni
        simulazione al variare di $dt$ come il massimo tra gli errori a
        varie slice temporali durante ogni simulazione.

    Per simulazioni con tempi molto lunghi i 2 modi restituiscono
    risultati simili.

```{=html}
<!-- -->
```
-   sono state effettuate diverse simulazioni per confrontare diverse
    discretizzazioni spaziali. La griglia di partenza è stata suddivisa
    in $N =30,\hspace{2}50,\hspace{2}80,\hspace{2} 100$ e i risultati
    ottenuti mostrano comportamenti analoghi per l'andamento degli
    errori.

![Errori per diversi valori di $N$, dove si è valutata la differenza di
varie slice temporali per ogni valore di $dt$ e se n'è considerato il
massimo. Sull'asse $x$ è riportato $d\tau=dt_i-dt_0$ con $dt_0$
spaziatura temporale
minima.](images/errore_vari_N_max_slice.png){#error30 width="15.5cm"}

![Errori per diversi valori di $N$, dove si è confrontato l'errore
relativo all'ultima slice temporale dopo lunghi tempi di evoluzione.
Sull'asse $x$ è riportato $d\tau=dt_i-dt_0$ con $dt_0$ spaziatura
temporale minima.](images/errore_vari_N_ultima_slice.png){#error100
width="15.5cm"}

# Algoritmo di Thomas

L'algoritmo di Thomas viene utilizzato per risolvere sistemi di
equazioni tridiagonali ed è un'alternativa al metodo di riduzione di
Gauss. Rispetto a quest'ultimo, che conta un numero $O(N^3)$ di step,
dove $N$ è la dimensione della matrice tridiagonale, l'algoritmo di
Thomas prevede $O(N)$ steps. Si procede come segue: dato un sistema di
equazioni tridiagonali nella forma $$\begin{bmatrix}
   b_1 & c_1 &        &        &  0      \\
   a_2 & b_2 & c_2    &        &         \\
       & a_3 & b_3    & \ddots &         \\
       &     & \ddots & \ddots & c_{N-1} \\
   0   &     &        & a_N    & b_N
\end{bmatrix}
    \begin{bmatrix}
    x_1\\
    x_2\\
    x_3\\
    ...\\
    x_N
    \end{bmatrix}
    =
    \begin{bmatrix}
    s_1\\
    s_2\\
    s_3\\
    ...\\
    s_N
    \end{bmatrix}$$ Il sistema di equazioni si può scrivere come:
$$a_i x_{i - 1}  + b_i x_i  + c_i x_{i + 1}  = s_i \quad \text{con} \quad a_1 = c_N = 0$$
I coefficienti sono modificati nel seguente modo *(forward
substitution)*: $$c'_i=
    \begin{cases}
    \frac{c_i}{b_i}\qquad i=1\\
    \frac{c_i}{b_i-a_{i}c'_{i-1}} \qquad i=2,..,N-1
    \end{cases}$$ e\
$$s'_i=
    \begin{cases}
    \frac{s_i}{b_i}\qquad i=1\\
    \frac{s_i-a_i s'_{i-1}}{b_i-a_i c'_{i-1}}\qquad i=2, ..., N
    \end{cases}$$ le soluzioni del sistema sono ottenute sostituendo i
nuovi coefficienti (*backward substitution*): $$x_N=s'_N$$
$$x_i=s'_i-c'_i x_{i+1}\qquad i=N-1, N-2,...,1.$$

::: thebibliography
99 V. A. Galaktionov J. L. Vazquez: A Stability Technique for Evolution
Partial Differential Equations, *Birkhauser Boston*, 2004

J. Naprstek & R. Kràl: Multi-dimensional Fokker-Planck equation analysis
using the modified finite element method, *J. Phys.: Conf. Ser. **744**
012177*, 2016

R. Mannella: PDEs, Introduction to Numerical Analysis, 2008 *appunti
lezioni*

J. W. Thomas: Numerical Partial Differential Equations, *Springer
Science+Business Media*, 1995
:::
